In [1]:
import os

In [2]:
%pwd

'c:\\Users\\nico_\\Desktop\\MLOPS\\datascienceproject\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\nico_\\Desktop\\MLOPS\\datascienceproject'

In [71]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir:Path
    train_data_path:Path
    test_data_path:Path
    model_name:str
    alpha:float
    l1_ratio:float
    target_column:str
    #separator : str

In [72]:
from src.datascienceproject.constants import *
from src.datascienceproject.utils.common import read_yaml, create_directories

In [73]:
class ConfigurationManager:
    def __init__(self,
                  config_filepath=CONFIG_FILE_PATH,
                  params_filepath=PARAMS_FILE_PATH,
                  schema_filepath=SCHEMA_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN
        #separator = self.params["data_ingestion"]["separator"]

        create_directories([self.config.artifacts_root])

        model_trainer_config=ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column = schema.name,
            #separator=separator
        )
        return model_trainer_config

In [74]:
from src.datascienceproject import logger
import pandas as pd
from sklearn.linear_model import ElasticNet
import joblib

In [78]:
class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig):
        self.config=config
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)#sep=self.config.separator)
        test_data = pd.read_csv(self.config.test_data_path)#sep=self.config.separator)

        train_x=train_data.drop([self.config.target_column], axis=1)
        test_x=test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x,train_y)

        os.makedirs(self.config.root_dir, exist_ok=True)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


In [79]:
try:
    config=ConfigurationManager()
    model_trainer_config=config.get_model_trainer_config()
    model_trainer=ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-05-02 12:55:28,449: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-02 12:55:28,451: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-02 12:55:28,452: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-02 12:55:28,455: INFO: common: created directory at: artifacts]
[2025-05-02 12:55:28,457: INFO: common: created directory at: artifacts]


In [77]:
data_train = pd.read_csv("artifacts/data_transformation/train.csv")
data_train.columns


Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')